In [1]:
# import modules
import copy
import time
from sys import path
from collections import defaultdict

In [2]:
import pandas as pd
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split

In [3]:
path.append('../code/src')

In [4]:
import svd_constraint
from post_rec import PostRec
from inter_rec import InterRec
from evaluate import Evaluation as ev

In [5]:
# constants
# file path
input_path = "../data/"
output_path = '../result/'
rate_file = 'reduced_rating_data.csv'
attr_file = 'recipe_data.csv'

# user/item max
user_max = 1000
food_max = 5000

# const count
const_count = 3

In [6]:
# Dictionary keys
rs1 = 'w/o Constraint'
rs2 = 'Post-Single-'
rs3 = 'Inter-Single-'
rs4 = 'Post-Mixed'
rs5 = 'Inter-Mixed'
val_r1 = 'RMSE'
val_r2 = 'nDCG'
val_t1 = 'train time(s)'
val_t2 = 'exec time(s)'

In [7]:
val_dict = {
    val_r1: 0,
    val_r2: 0,
    val_t1: 0,
    val_t2: 0
}

keys = [rs1]
keys = keys + [rs2 + str(i) for i in range (1, 4)]
keys = keys + [rs3 + str(i) for i in range (1, 4)]
keys = keys + [rs4]
keys = keys + [rs5]

result = {key: copy.deepcopy(val_dict) for key in keys}

In [14]:
# read previous result
result_df = pd.read_csv(output_path + 'evaluation_single.csv', index_col = 0).transpose()
result.update(result_df.to_dict())
result

{'w/o Constraint': {'RMSE': 0.706073395020865,
  'nDCG': 0.0461308504465657,
  'train time(s)': 0.2697818279266357,
  'exec time(s)': 14.747427940368652},
 'Post-Single-1': {'RMSE': 4.5924920826302165,
  'nDCG': 0.2149249591323901,
  'train time(s)': 0.2659465471903483,
  'exec time(s)': 250.6309250195821},
 'Post-Single-2': {'RMSE': 0.9891677322380804,
  'nDCG': 0.0553929970168118,
  'train time(s)': 0.2662857373555501,
  'exec time(s)': 69.36238193511963},
 'Post-Single-3': {'RMSE': 0.8392288178488778,
  'nDCG': 0.0658242065858615,
  'train time(s)': 0.2596391042073567,
  'exec time(s)': 176.79050389925638},
 'Inter-Single-1': {'RMSE': 3.8974948840491046,
  'nDCG': 0.2242529658185136,
  'train time(s)': 258.3454844156901,
  'exec time(s)': 10.339447975158691},
 'Inter-Single-2': {'RMSE': 0.927025030714022,
  'nDCG': 0.0613347070384644,
  'train time(s)': 268.81026641527814,
  'exec time(s)': 16.657366673151653},
 'Inter-Single-3': {'RMSE': 0.8274412369937522,
  'nDCG': 0.066672918515

# Evaluate Post-Rec w/ Mixed Type Constraint

In [8]:
def run_post(idx, k, save_result = True):    
    # PostRec applies constraint after the rating of each item is predicted
    rec = PostRec(input_path + rate_file, input_path + attr_file, 
                           input_path + 'const' + '.' + str(idx) + '.csv',
                            need_test = True)
    
    rec.get_data()  # get rating, attribute, recipe data
    rec.set_result_n(k)
    
    # train with data
    start = time.time()
    rec.train()  
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = rec.test_rmse()
    r1 = ev.calculate_rmse(predict_test)
       
    # get top-n for anti-test-set
    start = time.time()
    rec.test()
    top_n_df = rec.get_top_n()
    t2 = time.time() - start
    
    # calculate ndcg
    r2 = ev.calculate_ndcg(rec.get_rel(), top_n_df, k)
    
    if save_result:       
        top_n_df.to_csv(output_path + 'PostRec' + '.' + str(idx) + '.csv')
        
    return r1, r2, t1, t2

In [9]:
r1_sum = 0
r2_sum = 0
t1_sum = 0
t2_sum = 0
for j in range (1, const_count + 1):
    r1, r2, t1, t2 = run_post(j, 500, False)
    r1_sum = r1_sum + r1
    r2_sum = r2_sum + r2
    t1_sum = t1_sum + t1
    t2_sum = t2_sum + t2
    print('Const'+'.'+str(j)+" done")

result[rs4][val_r1] = r1_sum/const_count
result[rs4][val_r2] = r2_sum/const_count
result[rs4][val_t1] = t1_sum/const_count
result[rs4][val_t2] = t2_sum/const_count

Const.1 done
Const.2 done
Const.3 done


# Evaluate Inter-Rec w/ Mixed Type Constraint

In [10]:
def run_inter(idx, k, save_result = True):    
    # InterRec applies constraint during the rating of each item is predicted
    rec = InterRec(input_path + rate_file, input_path + attr_file, 
                           input_path + 'const' + '.' + str(idx) + '.csv', 
                            svd_constraint.CnstSVD(), need_test = True)
    
    rec.get_data()  # get rating, attribute, recipe data
    rec.set_result_n(k)
    
    # train with data
    start = time.time()
    rec.train()  
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = rec.test_rmse()
    r1 = ev.calculate_rmse(predict_test)
    
    # get top-n for anti-test-set
    start = time.time()
    rec.test()
    top_n_df = rec.get_top_n()
    t2 = time.time() - start

    # calculate nDCG
    r2 = ev.calculate_ndcg(rec.get_rel(), top_n_df, k)
    
    if save_result:       
        top_n_df.to_csv(output_path + 'InterRec' + '.' + str(idx) + '.csv')
        
    return r1, r2, t1, t2

In [11]:
r1_sum = 0
r2_sum = 0
t1_sum = 0
t2_sum = 0
for j in range (1, const_count + 1):
    r1, r2, t1, t2 = run_inter(j, 500, False)
    r1_sum = r1_sum + r1
    r2_sum = r2_sum + r2
    t1_sum = t1_sum + t1
    t2_sum = t2_sum + t2
    print('Const'+'.'+str(j)+" done")

result[rs5][val_r1] = r1_sum/const_count
result[rs5][val_r2] = r2_sum/const_count
result[rs5][val_t1] = t1_sum/const_count
result[rs5][val_t2] = t2_sum/const_count

Const.1 done
Const.2 done
Const.3 done


# Result

In [15]:
result_df = pd.DataFrame.from_dict(result, orient='index', columns=['RMSE', 'nDCG', 'train time(s)', 'exec time(s)'])
result_df.to_csv(output_path + 'evaluation_mixed.csv')
result_df

,RMSE,nDCG,train time(s),exec time(s)
w/o Constraint,0.706073,0.046131,0.269782,14.747428
Post-Single-1,4.592492,0.214925,0.265947,250.630925
Post-Single-2,0.989168,0.055393,0.266286,69.362382
Post-Single-3,0.839229,0.065824,0.259639,176.790504
Inter-Single-1,3.897495,0.224253,258.345484,10.339448
Inter-Single-2,0.927025,0.061335,268.810266,16.657367
Inter-Single-3,0.827441,0.066673,381.436334,19.531564
Post-Mixed,3.184275,0.080099,0.262108,198.562742
Inter-Mixed,2.755602,0.078789,289.832485,13.684747
